In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time

class GestureController:
    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.5
        )
        self.mp_draw = mp.solutions.drawing_utils
        self.background_color = (0, 0, 0)
        self.object_position = [320, 240]
        self.scene = 0
        self.palm_times = []  # Lista para almacenar los tiempos de palma abierta
        self.is_mirrored = False  # Estado del modo espejo
        self.show_object = False  # Estado de visibilidad del objeto
        
    def count_fingers(self, hand_landmarks):
        finger_tips = [8, 12, 16, 20]  # Índices de las puntas de los dedos (excepto pulgar)
        thumb_tip = 4
        count = 0
        
        if hand_landmarks.landmark[thumb_tip].x < hand_landmarks.landmark[thumb_tip - 1].x:
            count += 1
            
        for tip in finger_tips:
            if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y:
                count += 1
                
        return count

    def check_palm_gesture(self):
        current_time = time.time()
        self.palm_times.append(current_time)
        
        # Mantener solo los últimos 3 segundos
        self.palm_times = [t for t in self.palm_times if current_time - t <= 3]
        
        if len(self.palm_times) >= 3:
            self.is_mirrored = not self.is_mirrored
            self.palm_times = []  # Reiniciar contador
            return True
        return False
    
    def calculate_distance(self, p1, p2, hand_landmarks):
        x1, y1 = hand_landmarks.landmark[p1].x, hand_landmarks.landmark[p1].y
        x2, y2 = hand_landmarks.landmark[p2].x, hand_landmarks.landmark[p2].y
        return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    
    def process_frame(self, frame):
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(frame_rgb)
        
        if results.multi_hand_landmarks:
            for idx, (hand_landmarks, handedness) in enumerate(zip(results.multi_hand_landmarks, results.multi_handedness)):
                # Dibujar landmarks
                self.mp_draw.draw_landmarks(frame, hand_landmarks, self.mp_hands.HAND_CONNECTIONS)
                
                # Contar dedos y calcular distancia
                fingers = self.count_fingers(hand_landmarks)
                pinch_distance = self.calculate_distance(4, 8, hand_landmarks)
                
                # Obtener coordenadas de índice y pulgar
                thumb_x = int(hand_landmarks.landmark[4].x * frame.shape[1])
                thumb_y = int(hand_landmarks.landmark[4].y * frame.shape[0])
                index_x = int(hand_landmarks.landmark[8].x * frame.shape[1])
                index_y = int(hand_landmarks.landmark[8].y * frame.shape[0])
                
                # Verificar si los otros dedos están cerrados (gesto de "tamaño")
                other_fingers_closed = True
                for tip in [12, 16, 20]:  # Dedos medio, anular y meñique
                    if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y:
                        other_fingers_closed = False
                        break
                
                # Verificar alineación vertical (con margen de error)
                x_difference = abs(thumb_x - index_x)
                margin_of_error = frame.shape[1] * 0.05  # 5% del ancho de la imagen como margen de error
                
                # Solo mostrar la línea y distancia si los dedos están alineados verticalmente
                if other_fingers_closed and x_difference < margin_of_error:
                    # Dibujar línea entre índice y pulgar
                    cv2.line(frame, (thumb_x, thumb_y), (index_x, index_y), (0, 255, 0), 2)
                    
                    # Calcular punto medio para mostrar la distancia
                    mid_x = (thumb_x + index_x) // 2
                    mid_y = (thumb_y + index_y) // 2
                    
                    # Mostrar distancia
                    cv2.putText(frame, f"{pinch_distance:.2f}", (mid_x, mid_y), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                
                # Mostrar número de dedos en la esquina superior
                y_pos = 30 + (idx * 60)
                hand_text = f"{'Izq' if handedness.classification[0].label == 'Left' else 'Der'}: {fingers} dedos"
                cv2.putText(frame, hand_text, (10, y_pos), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        return frame

def main():
    cap = cv2.VideoCapture(3)     
    controller = GestureController()

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue
            
        frame = controller.process_frame(frame)
        cv2.imshow('Gesture Control', frame)
        
        if cv2.waitKey(1) & 0xFF == 27:  # Presionar ESC para salir
            break
            
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main() 